# 支持向量机原理(二) 线性支持向量机的软间隔最大化模型

在[支持向量机原理(一) 线性支持向量机](https://tianchi.aliyun.com/notebook/detail.html?spm=5176.11409386.0.0.4a3a1d07dvtRtm&id=23936)中，我们对线性可分SVM的模型和损失函数优化做了总结。最后我们提到了有时候不能线性可分的原因是线性数据集里面多了少量的异常点，由于这些异常点导致了数据集不能线性可分，本篇就对线性支持向量机如何处理这些异常点的原理方法做一个总结。

## 1. 线性分类SVM面临的问题

有时候本来数据的确是可分的，也就是说可以用 线性分类SVM的学习方法来求解，但是却因为混入了异常点，导致不能线性可分，比如下图，本来数据是可以按下面的实线来做超平面分离的，可以由于一个橙色和一个蓝色的异常点导致我们没法按照上一篇线性支持向量机中的方法来分类。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537424192582_KhIxTwbSiB.jpg)

另外一种情况没有这么糟糕到不可分，但是会严重影响我们模型的泛化预测效果，比如下图，本来如果我们不考虑异常点，SVM的超平面应该是下图中的红色线所示，但是由于有一个蓝色的异常点，导致我们学习到的超平面是下图中的粗虚线所示，这样会严重影响我们的分类模型预测效果。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537424229385_t3bA3Qxg9R.jpg)

如何解决这些问题呢？SVM引入了软间隔最大化的方法来解决。

## 2. 线性分类SVM的软间隔最大化

所谓的软间隔，是相对于硬间隔说的，我们可以认为上一篇线性分类SVM的学习方法属于硬间隔最大化。

回顾下硬间隔最大化的条件：

$$min \; \frac{1}{2}||w||^2_2 \; s.t \; y_i(w^Tx_i+b)≥1 \; (i=1,2,...m)$$

接着我们再看如何可以软间隔最大化呢？

SVM对训练集里面的每个样本$$(x_i,y_i)$$引入了一个松弛变量$$ξ_i≥0$$,使函数间隔加上松弛变量大于等于1，也就是说：

$$y_i(w∙x_i+b)≥1−ξ_i$$

对比硬间隔最大化，可以看到我们对样本到超平面的函数距离的要求放松了，之前是一定要大于等于1，现在只需要加上一个大于等于0的松弛变量能大于等于1就可以了。当然，松弛变量不能白加，这是有成本的，每一个松弛变量$$\xi_i$$,对应了一个代价 $$\xi_i$$，这个就得到了我们的软间隔最大化的SVM学习条件如下：

$$min\frac{1}{2}||w||_2^2 + C\displaystyle\sum_{i=1}^{m}ξi$$

$$s.t. \; y_i(w^Tx_i+b)≥1−ξ_i(i=1,2,...m)$$

$$ξ_i≥0(i=1,2,...m)$$

这里,C>0为惩罚参数，可以理解为我们一般回归和分类问题正则化时候的参数。C越大，对误分类的惩罚越大，C越小，对误分类的惩罚越小。

也就是说，我们希望$$\frac{1}{2}||w||^2_2$$尽量小，误分类的点尽可能的少。$$C$$是协调两者关系的正则化惩罚系数。在实际应用中，需要调参来选择。

这个目标函数的优化和上一篇的线性可分SVM的优化方式类似，我们下面就来看看怎么对线性分类SVM的软间隔最大化来进行学习优化。

## 3. 线性分类SVM的软间隔最大化目标函数的优化

和线性可分SVM的优化方式类似，我们首先将软间隔最大化的约束问题用拉格朗日函数转化为无约束问题如下：

$$L(w,b,ξ,α,μ)=\frac{1}{2}||w||_2^2+C\displaystyle\sum_{i=1}^{m}ξi−\displaystyle\sum_{i=1}^{m}α_i[y_i(w^Tx_i+b)−1+ξ_i]−\displaystyle\sum_{i=1}^{m}μ_iξ_i$$

其中$$μ_i≥0,α_i≥0$$,均为拉格朗日系数。

也就是说，我们现在要优化的目标函数是：

$$\underbrace{min}_{w,b,ξ} \underbrace{max}_{α_i≥0,μ_i≥0,}L(w,b,α,ξ,μ)$$

这个优化目标也满足KKT条件，也就是说，我们可以通过拉格朗日对偶将我们的优化问题转化为等价的对偶问题来求解如下：

$$\underbrace{max}_{α_i≥0,μ_i≥0,}\underbrace{min}_{w,b,ξ} \; L(w,b,α,ξ,μ)$$

我们可以先求优化函数对于$$w,b,ξ$$的极小值, 接着再求拉格朗日乘子$$α$$和$$μ$$的极大值。

首先我们来求优化函数对于$$w,b,ξ$$的极小值，这个可以通过求偏导数求得：

$$\frac{∂L}{∂w}=0⇒w=\displaystyle\sum_{i=1}^{m}α_iy_ix_i$$

$$\frac{∂L}{∂b}=0⇒\displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$\frac{∂L}{∂ξ}=0⇒C−α_i−μ_i=0$$

好了，我们可以利用上面的三个式子去消除$$w$$和$$b$$了。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537426154564_hBdoMd47m5.jpg)

其中，
(1)式到(2)式用到了$$C−α_i−μ_i=0$$,

(2)式到(3)式合并了同类项，

(3)式到(4)式用到了范数的定义$$||w||^2_2=w^Tw$$, 

(4)式到(5)式用到了上面的$$w=\displaystyle\sum_{i=1}^{m}
α_iy_ix_i$$， 

(5)式到(6)式把和样本无关的$$w^T$$提前，

(6)式到(7)式合并了同类项，

(7)式到(8)式把和样本无关的b提前，

(8)式到(9)式继续用到$$w=\displaystyle\sum_{i=1}^{m}
α_iy_ix_i$$，

(9)式到(10)式用到了向量的转置。由于常量的转置是其本身，所有只有向量$$x_i$$被转置,

(10)式到(11)式用到了上面的$$\displaystyle\sum_{i=1}^{m}
α_iy_i=0$$，

(11)式到(12)式使用了$$(a+b+c+…)(a+b+c+…)=aa+ab+ac+ba+bb+bc+…$$的乘法运算法则，

(12)式到(13)式仅仅是位置的调整。

仔细观察可以发现，这个式子和我们上一篇线性可分SVM的一样。唯一不一样的是约束条件。现在我们看看我们的优化目标的数学形式：

$$\underbrace{max}_{α}\displaystyle\sum_{i=1}^{m}α_i−\frac{1}{2}\displaystyle\sum_{i=1,j=1}^{m}α_iα_jy_iy_jx^T_ix_j$$

$$s.t. \; \displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$C−α_i−μ_i=0$$

$$α_i≥0(i=1,2,...,m)$$

$$μ_i≥0(i=1,2,...,m)$$

对于$$C−α_i−μ_i=0，α_i≥0，μ_i≥0$$这3个式子，我们可以消去$$μ_i$$，只留下$$αi$$，也就是说$$0≤α_i≤C$$。 同时将优化目标函数变号，求极小值，如下：

$$\underbrace{min}_{α} \frac{1}{2}\displaystyle\sum_{i=1,j=1}^{m}α_iα_jy_iy_jx^T_ix_j-\displaystyle\sum_{i=1}^{m}α_i$$

$$s.t. \; \displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$0≤α_i≤C$$

这就是软间隔最大化时的线性可分SVM的优化目标形式，和上一篇的硬间隔最大化的线性可分SVM相比，我们仅仅是多了一个约束条件$$0≤α_i≤C$$。我们依然可以通过SMO算法来求上式极小化时对应的α向量就可以求出w和b了。




## 4. 软间隔最大化时的支持向量

在硬间隔最大化时，支持向量比较简单，就是满足$$y_i(w^Tx_i+b)−1=0$$就可以了。

根据KKT条件中的对偶互补条件$$α^∗_i(y_i(w^Tx_i+b)−1)=0$$，

如果$$\alpha^∗_i>0$$ 

则有 $$y_i(w^Tx_i+b)=1$$ 即点在支持向量上，

否则如果$$\alpha^∗_i=0$$

则有$$y_i(w^Tx_i+b)≥1$$，即样本在支持向量上或者已经被正确分类。

在软间隔最大化时，则稍微复杂一些，因为我们对每个样本$$(x_i,y_i)$$引入了松弛变量$$ξ_i$$。

我们从下图来研究软间隔最大化时支持向量的情况，第i个点到对应类别支持向量的距离为$$ξ_i||w||_2$$。根据软间隔最大化时KKT条件中的对偶互补条件$$α^∗_i(y_i(w^Tx_i+b)−1+ξ^∗_i$$)=0我们有：

a) 如果$$α=0$$,那么$$y_i(w^Tx_i+b)−1≥0$$,即样本在间隔边界上或者已经被正确分类。如图中所有远离间隔边界的点。

b) 如果$$0 \lt \alpha \lt C$$,那么$$ξ_i=0,y_i(w^Tx_i+b)−1=0$$,即点在间隔边界上。

c) 如果$$α=C$$，说明这是一个可能比较异常的点，需要检查此时$$ξ_i$$
    
i)如果$$0≤ξ_i≤1$$,那么点被正确分类，但是却在超平面和自己类别的间隔边界之间。如图中的样本2和4.

ii)如果$$ξ_i=1$$,那么点在分离超平面上，无法被正确分类。

iii)如果$$ξ_i>1$$,那么点在超平面的另一侧，也就是说，这个点不能被正常分类。如图中的样本1和3.

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537428276730_QRTHTbWDmT.jpg)

## 5. 软间隔最大化的线性可分SVM的算法过程

这里我们对软间隔最大化时的线性可分SVM的算法过程做一个总结。

输入是线性可分的m个样本$$(x_1,y_1),(x_2,y_2),...,(x_m,y_m)$$,,其中$$x$$为$$n$$维特征向量。$$y$$为二元输出，值为1，或者-1.

输出是分离超平面的参数w∗和b∗和分类决策函数。

算法过程如下：

1）选择一个惩罚系数$$C>0$$, 构造约束优化问题

$$\underbrace{min}_{α} \frac{1}{2}\displaystyle\sum_{i=1,j=1}^{m}α_iα_jy_iy_jx^T_ix_j-\displaystyle\sum_{i=1}^{m}α_i$$

$$s.t. \; \displaystyle\sum_{i=1}^{m}α_iy_i=0$$

$$0≤α_i≤C$$

2）用SMO算法求出上式最小时对应的α向量的值α∗向量.

3) 计算$$w^∗=\displaystyle\sum_{i=1}^{m}α^*_iy_ix_i$$

4) 找出所有的S个支持向量,即满足$$0<α_s<C$$

对应的样本$$(x_s,y_s)$$，通过 $$y_s(\displaystyle\sum_{i=1}^{S}α_iy_ix^T_ix_s+b)=1$$，计算出每个支持向量$$(x_x,y_s)$$对应的$$b^∗_s$$,

计算出这些
$$b^∗_s=y_s−\displaystyle\sum_{i=1}^{S}α_iy_ix^T_ix_s$$. 

所有的$$b^∗_s$$对应的平均值即为最终的

$$b^∗=\frac{1}{S}\displaystyle\sum_{i=1}^{S}b^∗_s$$

这样最终的分类超平面为：$$w^∗∙x+b^∗=0$$，最终的分类决策函数为：$$f(x)=sign(w^∗∙x+b^∗)$$

## 6. 合页损失函数


线性支持向量机还有另外一种解释如下：

$$\underbrace{min}_{w,b}[1−y_i(w∙x+b)]++λ||w||^2_2$$
　

其中$$L(y(w∙x+b))=[1−y_i(w∙x+b)]_+$$称为合页损失函数(hinge loss function)，下标+表示为：

$$[z]_+ = \begin{cases}
   z & z>0 \\
   0 & z \le 0
\end{cases}$$

也就是说，

如果点被正确分类，且函数间隔大于1，损失是0，否则损失是$$1−y(w∙x+b)$$,如下图中的绿线。我们在下图还可以看出其他各种模型损失和函数间隔的关系：对于0-1损失函数，

如果正确分类，损失是0，误分类损失1， 如下图黑线，可见0-1损失函数是不可导的。对于感知机模型，感知机的损失函数是$$[−y_i(w∙x+b)]_+$$，

这样当样本被正确分类时，损失是0，误分类时，损失是$$−y_i(w∙x+b)$$，如下图紫线。对于逻辑回归之类和最大熵模型对应的对数损失，损失函数是$$log[1+exp(−y(w∙x+b))]$$, 如下图红线所示。

![](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/null/1537429372324_WFIzFvBxIa.jpg)

线性可分SVM通过软间隔最大化，可以解决线性数据集带有异常点时的分类处理，但是现实生活中的确有很多数据不是线性可分的，这些线性不可分的数据也不是去掉异常点就能处理这么简单。那么SVM怎么能处理中这样的情况呢？我们在下一篇就来讨论线性不可分SVM和核函数的原理。

文章转载自:  [博客园:  刘建平Pinard  支持向量机原理(二) 线性支持向量机的软间隔最大化模型](https://www.cnblogs.com/pinard/p/6100722.html)